In [1]:
import cv2
import numpy as np
from skimage import io
from skimage.transform import resize
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
import pandas as pd
import re
from gtts import gTTS
import hgtk
from io import BytesIO
import webbrowser
import numpy as np

In [2]:
# 동화 및 아이한글 크롤링 자료 중 3글자인 것들만 추출하여 저장한 파일 불러오기.

number3 = open('3str.txt' ,encoding='utf-8').readlines()

In [3]:
n3 = []

for line in number3:
    replace_line = line.replace('\n','')
    n3.append(replace_line)

In [4]:
# 중복 제거
new_list = []
for v in n3:
    if v not in new_list:
        new_list.append(v)
print(len(new_list))

423


In [5]:
dc= []
for i in range(0, len(new_list)):
    dc.append(hgtk.text.decompose(new_list[i]))

In [6]:
# 'ㅂ','ㄷ' 된소리되기 문제 생성  (ㄱ,ㄷ)
select2="ㅂᴥㄷ"
hard_list= []
for select in dc:
    if select2 in select or "ㄷᴥㅂ" in select :
        hard_list.append(select)

In [7]:
print(hard_list)

['ㅈㅡㄹᴥㄱㅓㅂᴥㄷㅏᴥ', 'ㅊㅏᴥㄱㅏㅂᴥㄷㅏᴥ', 'ㄴㅗㄹᴥㄹㅏㅂᴥㄷㅏᴥ', 'ㅅㅏᴥㄴㅏㅂᴥㄷㅏᴥ', 'ㅁㅜᴥㅅㅓㅂᴥㄷㅏᴥ', 'ㄱㅗㄷᴥㅂㅏᴥㄹㅗᴥ']


In [8]:
cm = []
for i in range(0, len(hard_list)):
    cm.append(hgtk.text.compose(hard_list[i]))
print(cm)

['즐겁다', '차갑다', '놀랍다', '사납다', '무섭다', '곧바로']


In [9]:
import random

index = random.randint(0, len(cm) - 1)
print(cm[index])

놀랍다


In [10]:
model_text = cm[index]

In [11]:
tts = gTTS(text=cm[index], lang='ko')
tts.save("bskyvision.mp3")

In [12]:
webbrowser.open('bskyvision.mp3')

True

In [13]:
model = keras.models.load_model('hand_written_korean_classification.hdf5')
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 64)       

In [14]:
B = pd.read_table('./label.txt',header=None)
B = B.reset_index(0)

In [15]:
drawing = False
ix, iy = -1, -1

In [ ]:
def draw_circle(event, x, y, flags, param):
     global ix, iy, drawing, mode

     if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        ix, iy = x, y

     if event == cv2.EVENT_MOUSEMOVE:
          if drawing == True:
                cv2.circle(img, (x, y), 5, (255, 255, 255), -1)

     elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        cv2.circle(img, (x, y), 5, (255, 255, 255), -1)

def Answer(img):
    img = cv2.resize(img, dsize=(96, 32), interpolation=cv2.INTER_AREA)
    
    word1 = img[:,:32,:]
    word2 = img[:,32:64,:]
    word3 = img[:,64:,:]
    
    word1 = cv2.resize(word1, dsize=(32,32))
    word1 = 255 - word1
    word1 = word1.reshape(1,32,32,3)
    word1 = word1/255
    
    word2 = cv2.resize(word2, dsize=(32,32))
    word2 = 255 - word2
    word2 = word2.reshape(1,32,32,3)
    word2 = word2/255
    
    word3 = cv2.resize(word3, dsize=(32,32))
    word3 = 255 - word3
    word3 = word3.reshape(1,32,32,3)
    word3 = word3/255
 #   img = img.reshape(32, 32, 3)
  #  img = 255 - img
   # img = cv2.resize(img, dsize=(32, 32))

    
    #img = img.reshape(1,32,32,3)
    #img = img/255
   
    print('입력값 : ', B.iloc[int(model.predict_classes(word1)),1])
    a1 = B.iloc[int(model.predict_classes(word1)),1]
    print('입력값 : ', B.iloc[int(model.predict_classes(word2)),1])
    a2 = B.iloc[int(model.predict_classes(word2)),1]
    print('입력값 : ', B.iloc[int(model.predict_classes(word3)),1])
    a3 = B.iloc[int(model.predict_classes(word3)),1]
    answer = a1+a2+a3
    print('사용자답안 :', answer)
    if(model_text == answer):
        #print("정답입니다.")
        path = 'C:/Users/user/Desktop/answer.png'
        image = cv2.imread(path)
        cv2.imshow('Good',image)
    else :
        path = 'C:/Users/user/Desktop/fail.png'
        image1 = cv2.imread(path)
        cv2.imshow('No',image1)
    
img = np.zeros((300, 900, 3), np.uint8)
img[:,300,:]=255
img[:,600,:]=255
cv2.namedWindow('Gakduki')
cv2.setMouseCallback('Gakduki', draw_circle)


while(1):
     cv2.imshow('Gakduki', img)
     k = cv2.waitKey(1) & 0xFF

     if k == 27: # ESC-CLOSE
        break
     elif k == 13: 
          Answer(img)
          img = np.zeros((300, 900, 3), np.uint8)
          img[:,300,:]=255
          img[:,600,:]=255

#---------------------------------------
cv2.destroyAllWindows()

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
입력값 :  놀
입력값 :  쟁
입력값 :  다
사용자답안 : 놀쟁다
입력값 :  돌
입력값 :  머
입력값 :  리
사용자답안 : 돌머리
